In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
image_size = (32, 32)
batch_size = 32
num_classes = 10
epochs = 10
learning_rate = 0.001

In [3]:
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(75, 75, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
from tensorflow import keras
import cv2
import numpy as np

(train_dataset, train_labels), (test_dataset, test_labels) = keras.datasets.cifar10.load_data()

resized_train_dataset = []
for image in train_dataset:
    resized_image = cv2.resize(image, (75, 75))
    resized_train_dataset.append(resized_image)
resized_train_dataset = np.array(resized_train_dataset)

resized_test_dataset = []
for image in test_dataset:
    resized_image = cv2.resize(image, (75, 75))
    resized_test_dataset.append(resized_image)
resized_test_dataset = np.array(resized_test_dataset)

train_dataset = resized_train_dataset.astype('float32') / 255.0
test_dataset = resized_test_dataset.astype('float32') / 255.0

In [6]:
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

In [7]:
print(train_dataset.shape)

(50000, 75, 75, 3)


In [8]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, dataset, labels, batch_size, image_size):
        self.dataset = dataset
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return int(np.ceil(len(self.dataset) / self.batch_size))

    def __getitem__(self, index):
        batch_x = self.dataset[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        resized_images = []
        for image in batch_x:
            resized_image = cv2.resize(image, self.image_size)
            resized_images.append(resized_image)
        resized_images = np.array(resized_images)

        return resized_images / 255.0, batch_y

batch_size = 32
train_generator = DataGenerator(train_dataset, train_labels, batch_size, (75, 75))
test_generator = DataGenerator(test_dataset, test_labels, batch_size, (75, 75))

In [9]:
batch_size = 32
train_generator = DataGenerator(train_dataset, train_labels, batch_size, (75, 75))
test_generator = DataGenerator(test_dataset, test_labels, batch_size, (75, 75))

In [ ]:
model.fit_generator(
    generator=train_generator,
    epochs=50,
    validation_data=test_generator
)

Epoch 1/50


<ipython-input-10-bb11df01dedd>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


1563/1563 [==============================] - 44s 23ms/step - loss: 2.1594 - accuracy: 0.1846 - val_loss: 2.0738 - val_accuracy: 0.2187
Epoch 2/50
1563/1563 [==============================] - 35s 22ms/step - loss: 1.9685 - accuracy: 0.2655 - val_loss: 2.0396 - val_accuracy: 0.2371
Epoch 3/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.9170 - accuracy: 0.2823 - val_loss: 1.9126 - val_accuracy: 0.2843
Epoch 4/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8776 - accuracy: 0.2988 - val_loss: 1.8185 - val_accuracy: 0.3311
Epoch 5/50
1563/1563 [==============================] - 40s 25ms/step - loss: 1.8371 - accuracy: 0.3158 - val_loss: 1.7956 - val_accuracy: 0.3310
Epoch 6/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8148 - accuracy: 0.3273 - val_loss: 1.8439 - val_accuracy: 0.3207
Epoch 7/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.7991 - accuracy: 0.3320 - val_loss: 1.7385 - val_accura